In [ ]:
import os
import sqlite3

import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

tz = ZoneInfo("Asia/Shanghai")

now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

ROUND = 2
DB_PATH = "../db.sqlite3"
MATCH_RESULT_PATH = "match_result/"

In [ ]:
db = sqlite3.connect(DB_PATH)

mentors = pd.read_sql_query("SELECT * FROM mentor", db)[["id", "name"]]
mentor_groups = pd.read_sql_query("SELECT * FROM mentor_groups", db)
cur_matches = pd.read_sql_query("SELECT * FROM match", db)
cur_matches = cur_matches[cur_matches["discarded"] == 0]
mentor_match_count = cur_matches.groupby("mentor_id").size().to_dict()

mentors = mentors.merge(mentor_groups, left_on="id", right_on="mentor_id", how="left")[["id_x", "name", "group_id"]]
mentors["match_count"] = mentors["id_x"].map(lambda x: mentor_match_count.get(x, 0))
valid_mentors = mentors[mentors["group_id"] == 1]
valid_mentors

In [ ]:
matches = pd.read_csv(os.path.join(MATCH_RESULT_PATH, f"matched_pairs_{ROUND}.csv"))
matches["assigned"] = False

matches = matches.sample(frac=1, ignore_index=True)
matches

# Assign match to mentors

In [ ]:
# mentor_claims = {
#     "mentor_id": ["applicant_ids"],
# }

# mentor_max = {
#     "mentor_id": int,
# }


new_matches = []
mentor_match_count = { mentor_id: mentor_count for mentor_id, mentor_count in mentors[["id_x", "match_count"]].values}

In [ ]:
# handle priority match
for i, (id1, id2, _) in matches.iterrows():
    for mentor_id, applicant_ids in mentor_claims.items():
        if id1 in applicant_ids or id2 in applicant_ids and not matches.loc[i, "assigned"]:
            new_matches.append([id1, id2, mentor_id])
            mentor_match_count[mentor_id] += 1
            matches.loc[i, "assigned"] = True
            break


# handle normal match: assign to mentor with lowest current count (among those under limits)
for i, (id1, id2, _) in matches.iterrows():
    if matches.loc[i, "assigned"]:
        continue
    eligible = [
        mid for mid in mentor_match_count.keys()
        if mentor_match_count[mid] < mentor_max.get(mid, 999)
    ]
    if not eligible:
        continue
    chosen = min(eligible, key=lambda mid: mentor_match_count[mid])
    new_matches.append([id1, id2, chosen])
    mentor_match_count[chosen] += 1
    matches.loc[i, "assigned"] = True

mentor_match_count

In [ ]:
new_matches_df = pd.DataFrame(new_matches, columns=["applicant1_id", "applicant2_id", "mentor_id", ])
new_matches_df["name"] = "取个组名吧!"
new_matches_df["round"] = ROUND

default_applicant_status = "P" if ROUND == 1 else "A"
new_matches_df["applicant1_status"] = default_applicant_status
new_matches_df["applicant2_status"] = default_applicant_status

new_matches_df["discarded"] = False
new_matches_df["discard_reason"] = None


new_matches_df["created_at"] = now
new_matches_df["updated_at"] = now

new_matches_df

# Danger zone!

In [ ]:
# raise Exception("Stop here")

proceed = input("you are about to write to db, proceed? (yes/no)")
if proceed == "yes":
    new_matches_df.to_sql("match", db, if_exists="append", index=False)
    
    if ROUND == 2:
        all_applicants = matches["applicant1_id"].tolist() + matches["applicant2_id"].tolist()
        
        cursor = db.cursor()
        placeholders = ",".join("?" * len(all_applicants))
        cursor.execute("UPDATE applicant SET confirmed = 1 WHERE id IN ({})".format(placeholders), all_applicants)
        db.commit()
        cursor.close()